In [28]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pymongo

In [32]:
url = "http://www.cine21.com/rank/boxoffice/domestic_content"
header = {}
header['page'] = 1
header['genre'] = 'all'
header['period'] = '2019-04-08'

res = requests.post(url, data=header)

In [33]:
res.status_code

200

In [34]:
soup = bs(res.content, 'html.parser')
print(soup)

<ul class="boxoffice_list">
<li class="boxoffice_li">
<a href="/movie/info/?movie_id=54122">
<img alt="" class="thumb" src="http://image.cine21.com/resize/cine21/poster/2019/0417/11_38_55__5cb691bfbbfd0[X224,320].jpg"/>
<div class="mov_name">어벤져스: 엔드게임</div>
<div class="people_num">관객수<span class="bar">|</span>13,779,286</div>
<!-- 오버시 나오는 정보 -->
<div class="over">
<ul class="info">
<li><span class="tit">관객수</span>13,779,286</li>
<li><span class="tit">누적관객수</span>13,779,286</li>
<li><span class="tit">배급사</span>월트디즈니컴퍼니코리아(주)</li>
<li><span class="tit">개봉일</span>2019-04-24</li>
</ul>
</div>
<!-- //오버시 나오는 정보 -->
<!-- 순위 --><span class="grade num1">1</span>
</a>
</li>
<li class="boxoffice_li">
<a href="/movie/info/?movie_id=54366">
<img alt="" class="thumb" src="http://image.cine21.com/resize/cine21/poster/2019/1121/11_09_00__5dd5f1bca6765[X224,320].jpg"/>
<div class="mov_name">겨울왕국 2</div>
<div class="people_num">관객수<span class="bar">|</span>13,748,486</div>
<!-- 오버시 나오는 정보 -->
<div cla

In [110]:
movies = soup.select("ul.boxoffice_list li.boxoffice_li")
for index, movie in enumerate(movies):
    name = movie.select_one('a > div.mov_name').text
    audience = movie.select_one('div.people_num').text
    audience = int(audience.split("|")[1].replace(",",''))
    # ranking도 문자열이므로 int형으로 바꿔주기
    ranking = int(movie.select_one('a span.grade.num1').text)
    
    #재귀적 접속 링크 만들어주기
    links = movie.select_one('a')['href']
    links = f"http://www.cine21.com{links}"
    for link in links:
        get_movie_details(link)

    

In [109]:
# 영화 정보 추출 함수
def get_movie_details(html):
        res = requests.get(html)
        soup = bs(res.content, 'html.parser')
        movie_details = {}
        year = soup.select_one("div.mov_info p.sub_info span:nth-child(1)").text
        country = soup.select_one("div.mov_info p.sub_info span:nth-child(2)").text
        proper_age = soup.select_one("div.mov_info p.sub_info span:nth-child(3)").text
        genre = soup.select_one("div.mov_info p:nth-child(4) span:nth-child(1)").text
        runtime = soup.select_one("div.mov_info p:nth-child(4) span:nth-child(2)").text
        runtime = runtime.split(':')[1].strip()
        released_date = soup.select_one("div.mov_info p:nth-child(5) span").text
        released_date = released_date.split(':')[1].strip()
        directors = soup.select("div.mov_info p:nth-child(6) a")
        directors_list = []
        for director in directors:
            director = director.text
            directors_list.append(director)
        # 인덱스 -1은 'more'을 나타내므로 마지막 인덱스를 제외한 나머지만 리스트로 담기
        actors = soup.select("div.mov_info p:nth-child(7) a")[:-1]
        actors_list = []
        for actor in actors:
            actor = actor.text
            actors_list.append(actor)
        cine_reviews = float(soup.select_one("ul.mov_rating li:nth-child(1) span").text)
        people_reviews = float(soup.select_one("ul.mov_rating li:nth-child(2) span").text)

        movie_details['개봉연도'] = year
        movie_details['국가'] = country
        movie_details['관람적절연령'] = proper_age
        movie_details['장르'] = genre
        movie_details['상영시간'] = runtime
        movie_details['개봉날짜'] = released_date
        movie_details['감독'] = directors_list
        movie_details['출연배우'] = actors_list
        movie_details['씨네21평점'] = cine_reviews
        movie_details['네티즌평점'] = people_reviews

        print(movie_details)
